In [1]:
# Importación de paquetes requeridos
import os
import io
import requests
import pandas as pd
import matplotlib.pyplot as plt
plt.close("all")
import numpy as np

from IPython.display import display, Markdown, Latex

import json

In [2]:
import matplotlib.pylab as pylab
params = {
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
    'lines.linewidth' : 3,
}
pylab.rcParams.update(params)

In [3]:
# Función auxiliar para visualizar los datos
def pp(obj):
    print(json.dumps(obj, indent=2))

In [4]:
# La operación queda con la forma siguiente:
# «para cada fila r, convertir el valor de la columna "col" en un tipo numérico (pd.to_numeric),
# ignorando los valores nulos (errors='coerce'), redondeando hacia abajo al entero más cercano
# (np.floor) y almacenándolo como entero de 64 bits (astype('Int64'))»

def pd_to_int(df:pd.DataFrame, col:str) -> int:
    return np.floor(pd.to_numeric(df[col], errors='coerce')).astype('Int64')

In [5]:
# Parmámetros
iga_api_base_url = 'http://ing-acc-movil01.personal.corp/ingenier@/symfony/public/index.php/api'

In [6]:
display(Markdown(f"# Ejemplo 003: Descarga del reporte de Datos Outdoor y post procesado para uso del área de STEM"))

# Ejemplo 003: Descarga del reporte de Datos Outdoor y post procesado para uso del área de STEM

In [9]:
# Llamamos a un endpoint de prueba en la API de IGA para verificar conectividad
display(Markdown("### IGA API test"))
x = requests.get(iga_api_base_url + '/public/db-sync-state')
pp(x.json())

### IGA API test

{
  "status": "success",
  "data": {
    "Uid": "41337",
    "State": "DB_SYNC_STATE_IDLE",
    "LastSyncTimestamp": "01-06-2022 11:14:05",
    "SyncState": "OK",
    "ProjectsUpdated": "DataManager, AMBA, LINO"
  }
}


In [17]:
# Llamamos al endpoint generic-report para obtener el reporte de datos outdoor de los sitios operativos:

# Parámetros en el payload de la request con que llamamos al endpoint de IGA:
payload = {
    # Nombre del reporte: datos_outdoor
    "reportName": "datos_outdoor",
    # Sitios seleccionados: ninguno
    "siteNames": [],
    # Parámetros de filtrado: todos los sitios en estado operativo (OA).
    # Son los sitios que tienen al menos una celda operativa.
    "filteringParams": { "SiteVisualizationData.SiteStatus": ["OA"] },
    # Seleccionar por nombres de sitio: no
    "selectBySiteNames": False,
    # Seleccionar por parámetros de filtrado: sí
    "selectByFilteringParams": True
}
# Obtenemos los datos. Este proceso puede tardar algunos minutos.
x = requests.post(iga_api_base_url + '/public/report/generic-report', json=payload)

# En x almacenamos los datos devueltos por el endpoint de IGA.
# Es un archivo xlsx en formato binario

# Usando pandas, con la función read_excel, leemos los datos y los almacenamos
# en memoria como un objeto de python:
with io.BytesIO(x.content) as fh:
    df_raw = pd.read_excel(fh)


In [18]:
# Visualizamos los datos obtenidos:
display(Markdown(f"# Datos Outdoor - Sitios: SiteStatus = OA"))
display(Markdown(f"### Tabla con orientación normal:"))
display(df_raw)

display(Markdown(f"### Tabla transpuesta:"))
# Los mismos datos pero transponiéndolos (intercambiando filas por columnas) para poder ver mejor todos los nombres de columnas:
display(df_raw.transpose())

# Datos Outdoor - Sitios: SiteStatus = OA

### Tabla con orientación normal:

,Proyecto,Código,Sector,Portadora,Estado,Latitud Antena,Longitud Antena,Id Antena,Modelo Antena,Id Puerto,...,Altura Antena,Azimuth Físico,Azimuth Efectivo,Tilt Mec.,Tilt Eléc.,Twist,Ancho Horiz.,Ancho Vert.,Ganancia dBi,Cota de Terreno
0,MEDI,A01-E133,XRCESEL1G,LTE FDD_AWS_2225,Operativo,-33.142181,-64.348739,1,AAU3940,1,...,12.0,40.0,39.0,-0.0,2.0,0.0,73,12,13.972812,436.86
1,MEDI,A01-E133,XRCESEL1H,LTE FDD_AWS_2225,Operativo,-33.142181,-64.348739,2,AAU3940,1,...,12.0,160.0,159.0,-0.0,2.0,0.0,73,12,13.972812,436.86
2,MEDI,A01-E133,XRCESEL1I,LTE FDD_AWS_2225,Operativo,-33.142181,-64.348739,3,AAU3940,1,...,12.0,280.0,279.0,-0.0,2.0,0.0,73,12,13.972812,436.86
3,MEDI,A01-E133,XRCESEM1G,LTE FDD_1900MHz_975,No Operativo,-33.142181,-64.348739,1,AAU3940,1,...,12.0,40.0,39.0,-0.0,2.0,0.0,73,12,13.972812,436.86
4,MEDI,A01-E133,XRCESEM1H,LTE FDD_1900MHz_975,No Operativo,-33.142181,-64.348739,2,AAU3940,1,...,12.0,160.0,159.0,-0.0,2.0,0.0,73,12,13.972812,436.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175846,AMBA,SPU990,SBBORIU12,WCDMA1900_9788,Operativo,-34.790932,-58.239595,2,DBXDH-6565A-VTM,3,...,21.0,180.0,180.0,-0.0,2.0,0.0,64,10,15.331015,18.77
175847,AMBA,SPU990,SBBORIU13,WCDMA1900_9788,Operativo,-34.790932,-58.239595,3,DBXDH-6565A-VTM,3,...,21.0,300.0,300.0,-0.0,10.0,0.0,64,10,15.331015,18.77
175848,AMBA,SPU990,SBBORIV21,WCDMA850_4358,Operativo,-34.790932,-58.239595,1,DBXDH-6565A-VTM,1,...,21.0,60.0,60.0,-0.0,4.0,0.0,64,10,15.331015,18.77
175849,AMBA,SPU990,SBBORIV22,WCDMA850_4358,Operativo,-34.790932,-58.239595,2,DBXDH-6565A-VTM,1,...,21.0,180.0,180.0,-0.0,4.0,0.0,64,10,15.331015,18.77


### Tabla transpuesta:

,0,1,2,3,4,5,6,7,8,9,...,175841,175842,175843,175844,175845,175846,175847,175848,175849,175850
Proyecto,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,...,AMBA,AMBA,AMBA,AMBA,AMBA,AMBA,AMBA,AMBA,AMBA,AMBA
Código,A01-E133,A01-E133,A01-E133,A01-E133,A01-E133,A01-E133,A01-E133,A01-E133,A01-E133,A01-E134,...,SPU990,SPU990,SPU990,SPU990,SPU990,SPU990,SPU990,SPU990,SPU990,SPU990
Sector,XRCESEL1G,XRCESEL1H,XRCESEL1I,XRCESEM1G,XRCESEM1H,XRCESEM1I,XRCESEU1G,XRCESEU1H,XRCESEU1I,XRCEALL1G,...,SBBORIN13,SBBORIO11,SBBORIO12,SBBORIO13,SBBORIU11,SBBORIU12,SBBORIU13,SBBORIV21,SBBORIV22,SBBORIV23
Portadora,LTE FDD_AWS_2225,LTE FDD_AWS_2225,LTE FDD_AWS_2225,LTE FDD_1900MHz_975,LTE FDD_1900MHz_975,LTE FDD_1900MHz_975,WCDMA1900_9813,WCDMA1900_9813,WCDMA1900_9813,LTE FDD_AWS_2225,...,LTE FDD_700MHz_9360,LTE FDD_850MHz_2474,LTE FDD_850MHz_2474,LTE FDD_850MHz_2474,WCDMA1900_9788,WCDMA1900_9788,WCDMA1900_9788,WCDMA850_4358,WCDMA850_4358,WCDMA850_4358
Estado,Operativo,Operativo,Operativo,No Operativo,No Operativo,No Operativo,Operativo,Operativo,Operativo,Operativo,...,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo
Latitud Antena,-33.142181,-33.142181,-33.142181,-33.142181,-33.142181,-33.142181,-33.142181,-33.142181,-33.142181,-33.141588,...,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932
Longitud Antena,-64.348739,-64.348739,-64.348739,-64.348739,-64.348739,-64.348739,-64.348739,-64.348739,-64.348739,-64.338766,...,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595
Id Antena,1,2,3,1,2,3,1,2,3,1,...,6,1,2,3,1,2,3,1,2,3
Modelo Antena,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,...,ATR4518R13,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM
Id Puerto,1,1,1,1,1,1,1,1,1,1,...,3,1,1,1,3,3,3,1,1,1


In [19]:
# Primero filtramos por la columna 'Estado' para quedarnos únicamente con las celdas operativas:
df = df_raw.loc[df_raw['Estado'] == 'Operativo'].copy()

# Agregamos las columnas config_puertos_r y config_puertos_t, que almacenan la cantidad de puertos de
# Uplink y Downlink respectivamente. Para eso, usamos la función apply que obtiene para cada fila el
# valor de la nueva columna como una función de los demás valores de la fila.
# Por ejemplo, para el caso de config_puertos_r, tomamos la columna 'Id Puertos Uplink' que contiene
# los id de puertos separados por coma.
# Separamos los id usando la función split y el separador ',', con lo que convertimos el texto en un
# array con todos los elementos.
# Por último, almacenamos en la nueva columna la cantidad de elementos del array usando la función len:
df['config_puertos_r'] = df.apply(lambda r: len(r['Id Puertos Uplink'].split(',')), axis=1)
df['config_puertos_t'] = df.apply(lambda r: len(r['Id Puertos Downlink'].split(',')), axis=1)

# Algunos campos deben ser convertidos a enteros. Aplicamos un método similar al anterior, pero
# esta vez usamos una función definida anteriormente pd_to_int, que convierte el número en entero,
# o None (en python None significa valor vacío) en caso de estar vacío (Ej. el tilt eléctrico es
# nulo en celdas que no tienen ningún controlador eléctrico asignado)

df['Azimuth Efectivo'] = pd_to_int(df, 'Azimuth Efectivo')
df['Tilt Mec.'] = pd_to_int(df, 'Tilt Mec.')
df['Tilt Eléc.'] = pd_to_int(df, 'Tilt Eléc.')
df['Ancho Horiz.'] = pd_to_int(df, 'Ancho Horiz.')
df['Ancho Vert.'] = pd_to_int(df, 'Ancho Vert.')

display(df.transpose())

,0,1,2,6,7,8,9,10,11,15,...,175841,175842,175843,175844,175845,175846,175847,175848,175849,175850
Proyecto,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,...,AMBA,AMBA,AMBA,AMBA,AMBA,AMBA,AMBA,AMBA,AMBA,AMBA
Código,A01-E133,A01-E133,A01-E133,A01-E133,A01-E133,A01-E133,A01-E134,A01-E134,A01-E134,A01-E134,...,SPU990,SPU990,SPU990,SPU990,SPU990,SPU990,SPU990,SPU990,SPU990,SPU990
Sector,XRCESEL1G,XRCESEL1H,XRCESEL1I,XRCESEU1G,XRCESEU1H,XRCESEU1I,XRCEALL1G,XRCEALL1H,XRCEALL1I,XRCEALU1G,...,SBBORIN13,SBBORIO11,SBBORIO12,SBBORIO13,SBBORIU11,SBBORIU12,SBBORIU13,SBBORIV21,SBBORIV22,SBBORIV23
Portadora,LTE FDD_AWS_2225,LTE FDD_AWS_2225,LTE FDD_AWS_2225,WCDMA1900_9813,WCDMA1900_9813,WCDMA1900_9813,LTE FDD_AWS_2225,LTE FDD_AWS_2225,LTE FDD_AWS_2225,WCDMA1900_9813,...,LTE FDD_700MHz_9360,LTE FDD_850MHz_2474,LTE FDD_850MHz_2474,LTE FDD_850MHz_2474,WCDMA1900_9788,WCDMA1900_9788,WCDMA1900_9788,WCDMA850_4358,WCDMA850_4358,WCDMA850_4358
Estado,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,...,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo
Latitud Antena,-33.142181,-33.142181,-33.142181,-33.142181,-33.142181,-33.142181,-33.141588,-33.141588,-33.141588,-33.141588,...,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932,-34.790932
Longitud Antena,-64.348739,-64.348739,-64.348739,-64.348739,-64.348739,-64.348739,-64.338766,-64.338766,-64.338766,-64.338766,...,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595,-58.239595
Id Antena,1,2,3,1,2,3,1,2,3,1,...,6,1,2,3,1,2,3,1,2,3
Modelo Antena,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,AAU3940,...,ATR4518R13,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM,DBXDH-6565A-VTM
Id Puerto,1,1,1,1,1,1,1,1,1,1,...,3,1,1,1,3,3,3,1,1,1


In [20]:
# Finalmente creamos otro DataFrame seleccionando las columnas que nos interesan
# y renombrando los encabezados
df2 = df[[
    'Sector',
    'Latitud Antena',
    'Longitud Antena',
    'Azimuth Efectivo',
    'Tilt Mec.',
    'Tilt Eléc.',
    'Ancho Horiz.',
    'Ancho Vert.',
    'Ganancia dBi',
    'Cota de Terreno',
    'config_puertos_r',
    'config_puertos_t',
]]
df2.columns=[
    'celda',
    'latitud',
    'longitud',
    'azimuth_efvo',
    'tilt_mec',
    'tilt_elec',
    'ancho_horiz',
    'ancho_vert',
    'ganancia_dbi',
    'cota_terreno',
    'config_puertos_r',
    'config_puertos_t',
]
display(df2)

,celda,latitud,longitud,azimuth_efvo,tilt_mec,tilt_elec,ancho_horiz,ancho_vert,ganancia_dbi,cota_terreno,config_puertos_r,config_puertos_t
0,XRCESEL1G,-33.142181,-64.348739,39,0,2,73,12,13.972812,436.86,2,2
1,XRCESEL1H,-33.142181,-64.348739,159,0,2,73,12,13.972812,436.86,2,2
2,XRCESEL1I,-33.142181,-64.348739,279,0,2,73,12,13.972812,436.86,2,2
6,XRCESEU1G,-33.142181,-64.348739,39,0,2,73,12,13.972812,436.86,2,2
7,XRCESEU1H,-33.142181,-64.348739,159,0,2,73,12,13.972812,436.86,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
175846,SBBORIU12,-34.790932,-58.239595,180,0,2,64,10,15.331015,18.77,2,2
175847,SBBORIU13,-34.790932,-58.239595,300,0,10,64,10,15.331015,18.77,2,2
175848,SBBORIV21,-34.790932,-58.239595,60,0,4,64,10,15.331015,18.77,2,2
175849,SBBORIV22,-34.790932,-58.239595,180,0,4,64,10,15.331015,18.77,2,2


In [21]:
# Creamos la carpeta 'output' si no existe
os.makedirs('output', exist_ok=True)

# Exportamos los datos como .csv
df2.to_csv('output/ejemplo-003.csv', index=False)